## Parser Transitions

In [31]:
from parser_transitions import *

In [32]:
#init
sentence = ["I", "attended", "lectures", "in", "the", "NLP", "class"]
pp = PartialParse(sentence)

#parse
pp.parse_step("S")
pp.parse_step("S")
pp.parse_step("LA")
pp.parse_step("S")

#print
[pp.stack, pp.buffer, pp.dependencies]

[['ROOT', 'attended', 'lectures'],
 ['in', 'the', 'NLP', 'class'],
 [('attended', 'I')]]

In [33]:
#init
sentence = ["I", "attended", "lectures", "in", "the", "NLP", "class"]
pp = PartialParse(sentence)

#parse
transitions = ["S", "S", "LA", "S", "RA", "S", "S", "S", "S", "LA", "LA", "LA", "RA", "RA"]
pp.parse(transitions)

#print
[pp.stack, pp.buffer, pp.dependencies]

[['ROOT'],
 [],
 [('attended', 'I'),
  ('attended', 'lectures'),
  ('class', 'NLP'),
  ('class', 'the'),
  ('class', 'in'),
  ('attended', 'class'),
  ('ROOT', 'attended')]]

Minibatch parse

In [42]:
model = DummyModel()
sentence = ["I", "attended", "lectures", "in", "the", "NLP", "class"]
sentences = [sentence, "I love you".split()]    
pp = PartialParse(sentence)

In [43]:
transitions = model.predict([pp])

In [44]:
minibatch_parse(sentences, model, 1)

[[('class', 'NLP'),
  ('class', 'the'),
  ('class', 'in'),
  ('class', 'lectures'),
  ('class', 'attended'),
  ('class', 'I'),
  ('class', 'ROOT')],
 [('you', 'love'), ('you', 'I'), ('you', 'ROOT')]]

In [39]:
pp.buffer

['I', 'attended', 'lectures', 'in', 'the', 'NLP', 'class']